# 다중 슬롯머신 문제(multi armed bandit)

- 환경
    - 여러 손잡이 중 하나를 선택해 1달러를 넣고 당김
        - 그 결과 플레이어는 1달러를 잃거나 1달러를 따게 됨
    - 손잡이마다 승리할 확률이 정해져 있음
        - 돈을 따려면 확률이 높은 손잡이를 당겨야 함
    - 확률은 숨겨져 있음
    
- 플레이어가 취할 수 있는 행동
    - 행동에 따라 보상이 주어짐
    - 행동 : 손잡이를 고르는 일
    - 손잡이가 5개라면 행동의 잡합은 0, 1, 2, 3, 4
    
- 보상
    - 돈을 잃거나 따는 것
        - 1달러를 따거나 1달러를 잃거나
    - 보상의 집합은 1, -1
    
- 슬롯머신 문제의 특성
    - 다중 슬롯머신 문제에는 상태가 없음
    - 손잡이를 당긴 후에도 환경은 그대로임
        - 손잡이도 그대로, 손잡이마다 설정된 확률도 그대로
    - 다중 슬롯머신은 행동과 보상만 있는 단순한 문제임
        - 강화학습의 기초개념을 익히기에 적합

## 탐험형 정책과 탐사형 정책

- 탐험형 정책(exploration)
    - 처음부터 끝까지 손잡이를 무작위로 선택하는 랜덤 정책(random policy)
    - 이전 경험을 전혀 사용하지 않는 매우 비효율적인 방식
    
- 탐사형 정책(exploitation)
    - 몇 번 시도해보고 이후에는 그때까지 가장 높은 확률을 보인 손잡이만 계속 당기는 정책
    - 확률이 더 높은 손잡이를 놓칠 위험이 있음
    
- 위의 두 정책은 양 극단에 속함
- 강화학습에서는 둘 사이에서 균형을 잡는 것이 중요
    - 예) 현재까지 높은 확률을 보인 손잡이를 더 자주 당기지만 일정 비율로 다른 손잡이도 시도해보는 등
    - 이때 다른 손잡이를 시도하는 비율이 높을수록 탐형형에 가깝고 낮을수록 탐사형에 가까움
    
- 슬롯머신 문제는 단순하기 때문에 연속으로 행동을 취하고 마칠 때까지의 기록(에피소드)를 충분히 길게 하여 최적의 정책을 찾을 수 있음
    - 랜덤 정책을 길게 적용하여 충분히 긴 에피소드를 수집하고 손잡이마다 돈을 따게될 확률을 계산
        - 예) 1000달러를 가지고 랜덤 정책을 이용해 길이가 1000인 에피소드를 수집
        - 모든 손잡이에 대해 확률을 계산하여 네 번째 손잡이의 확률이 가장 높다면 수익을 최대로 보장하는 최적정책(optimal policy)는 (0, 0, 0, 1, 0)
        - 네 번째 손잡이를 당기는 확률을 1로 설정하면 많은 돈을 벌 수 있음
    - 하지만 실제로는 확률이 바뀔 수도 있고, 신뢰할만한 확률을 알아낼 수 있을 정도로 돈과 시간이 충분하지 않을 수 있기 때문에 더 영리한 알고리즘이 필요

### 랜덤 정책을 쓰는 알고리즘

- 알고리즘의 목표는 확률을 모른 채 플레이하면서 수익을 최대화하는 정책을 찾는 것
- 알고리즘에 주어지는 정보는 행동을 했을 때의 보상 뿐

In [1]:
import numpy as np

In [2]:
# 다중 손잡이 밴딧 문제 설정
arms_profit = [0.4, 0.12, 0.52, 0.6, 0.25]
n_arms = len(arms_profit)

n_trial = 10000 # 손잡이를 당기는 횟수(에피소드 길이)

In [3]:
# 손잡이를 당기는 행위를 시뮬레이션 하는 함수(handle은 손잡이 번호)
def pull_bandit(handle):
    q = np.random.random()
    if q < arms_profit[handle]:
        return 1
    else:
        return -1

In [4]:
# 랜덤 정책을 모방하는 함수
def random_exploration():
    episode = []
    num = np.zeros(n_arms) # 손잡이별로 당긴 횟수
    wins = np.zeros(n_arms) # 손잡이별로 승리 횟수
    
    for i in range(n_trial):
        h = np.random.randint(0, n_arms)
        reward = pull_bandit(h)
        episode.append([h, reward])
        num[h] += 1
        wins[h] += 1 if reward == 1 else 0
    return episode, (num, wins)

In [5]:
e, r = random_exploration()

print("손잡이별 승리 확률:", ["%6.4f"% (r[1][i]/r[0][i]) if r[0][i]>0 else 0.0 for i in range(n_arms)])
print("손잡이별 수익($)", ["%d"% (2 * r[1][i] - r[0][i]) for i in range(n_arms)])
print("순 수익($)", sum(np.asarray(e)[:, 1]))

손잡이별 승리 확률: ['0.4084', '0.1192', '0.5253', '0.5977', '0.2395']
손잡이별 수익($) ['-371', '-1571', '99', '401', '-992']
순 수익($) -2434


### ε-탐욕 알고리즘(greedy algorithm)

- 탐욕 알고리즘 : 과거와 미래를 전혀 고려하지 않고 현재 순간의 정보만으로 선택을 하는 알고리즘 방법론
    - 탐험형과 탐사형 중 탐사형에 치우친 방법론

- ε-탐욕 알고리즘
    - 기본적으로는 탐욕 알고리즘
    - ε비율만큼 탐험을 적용하여 탐험과 탐사의 균형을 추구
    - 현재까지 파악한 승리 확률에 따라 행동을 선택하는 탐사형 방식을 사용하지만, ε비율만큼 랜덤 정책을 적용하여 탐험형을 섞음

In [6]:
# ε-탐욕을 구현하는 함수
def epsilon_greedy(eps):
    episode = []
    num = np.zeros(n_arms)
    wins = np.zeros(n_arms)
    
    for i in range(n_trial):
        r = np.random.random()
        
        if (r < eps or sum(wins) == 0): # 확률 eps로 임의 선택 or 승리한적이 없다면
            h = np.random.randint(0, n_arms)
        else:
            prob = np.asarray([wins[i] / num[i] if num[i] > 0 else 0.0 for i in range(n_arms)])
            prob = prob/sum(prob)
            h = np.random.choice(range(n_arms), p = prob)
        reward = pull_bandit(h)
        episode.append([h, reward])
        num[h] += 1
        wins[h] +=1 if reward == 1 else 0
    return episode, (num, wins)

In [7]:
e, r = epsilon_greedy(0.1)

print("손잡이별 승리 확률:", ["%6.4f"% (r[1][i]/r[0][i]) if r[0][i]>0 else 0.0 for i in range(n_arms)])
print("손잡이별 수익($)", ["%d"% (2 * r[1][i] - r[0][i]) for i in range(n_arms)])
print("순 수익($)", sum(np.asarray(e)[:, 1]))

손잡이별 승리 확률: ['0.3916', '0.1240', '0.5053', '0.6051', '0.2736']
손잡이별 수익($) ['-452', '-564', '28', '631', '-687']
순 수익($) -1044


- 탐험과 탐사의 균형을 추구하는 엡실론-탐욕 알고리즘은 이전 정보를 완전히 무시하는 랜덤 정책에 비해 훨씬 영리함
- 현실 세계의 현상 또는 수학적 현상을 난수로 생성하여 시뮬레이션하는 기법을 통틀어 몬테카를로 방법(Monte Carlo method)라고 함
    - 인공지능은 강화학습 뿐만 아니라 다양한 목적으로 몬테카를로 방법을 활용
    - 랜덤 정책 알고리즘과 엡실론-탐욕 알고리즘을 사용한 다중 슬롯머신 문제는 다중 슬롯머신의 동작을 난수로 시뮬레이션했기 떄문에 몬테카를로 방법에 속함

# OpenAI의 gym 라이브러리

- 강화학습을 구현할 때는 OpenAI 재단이 제공하는 gym 라이브러리를 주로 사용
- OpenAI
    - 누구나 혜택을 받는 안전한 인공지능을 구현하려는 목표로 창립한 비영리 재단
- gym 라이브러리
    - 강화학습으로 풀어야 하는 문제를 여러 가지 제공
    - FrozenLake
        - 시작점에서 출발해 한 칸씩 움직여 목표 지점에 도달하면 이기는 게임
        - 중간에 구멍에 빠지면 지고, 밟고 지나갈 수 있는 곳은 정해져 있음
        - 에이전트에는 어떤 칸이 구멍이고 어떤 칸이 지나갈 수 있는지에 대한 정보는 주어지지 않음
    
    - CartPole
        - 막대를 오래 세워놓을수록 높은 점수를 받는 문제
        - 막대가 왼쪽으로 기울어지면 수레를 왼쪽으로 움직여 평형을 이루어야함
        - 너무 극단적으로 많이 이동하면 오히려 회복이 불가능해짐
        
    - MountainCar
        - 언덕에 있는 깃발에 도달하면 이기는 게임
        - 반대쪽 언덕으로 충분히 올라간 다음 구르는 힘으로 목적지에 도달해야함
        
    - FetchSlide
        - 로봇이 손으로 검은색 퍽을 쳐서 특정 지점으로 옮기는 문제
        - 정확한 방향을 향해 적절한 힘을 가해야 성공할 수 있음
        
    - Atari
        - 1980년대에 유행하던 고전 비디오게임 수십종이 제공됨
        - 대표적인 것은 Breakout, BankHeist, FishingDerby

# FrozenLake 문제

In [8]:
import gym

In [36]:
# 환경 불러오기
env = gym.make("FrozenLake-v1", is_slippery = False, render_mode = "human")
print(env.observation_space)
print(env.action_space)

n_trial = 20

# 에피소드 수집
env.reset()
episode = []
for i in range(n_trial):
    action = env.action_space.sample() # 가능한 액션 나열 및 선택
    obs, reward, terminated, truncated, info = env.step(action) # 액션에 따른 결과 출력
    episode.append([action, reward, obs])
    env.render()
    if terminated or truncated:
        observation, info = env.reset()
        
print(episode)
env.close()

Discrete(16)
Discrete(4)
[[1, 0.0, 4], [0, 0.0, 4], [1, 0.0, 8], [1, 0.0, 12], [2, 0.0, 1], [3, 0.0, 1], [0, 0.0, 0], [1, 0.0, 4], [3, 0.0, 0], [2, 0.0, 1], [0, 0.0, 0], [1, 0.0, 4], [2, 0.0, 5], [0, 0.0, 0], [3, 0.0, 0], [2, 0.0, 1], [2, 0.0, 2], [0, 0.0, 1], [0, 0.0, 0], [2, 0.0, 1]]


- Discrete(16)(observation_space)
    - 16개의 상태가 있다는 뜻
    - 4 x 4의 격자 위의 어느 곳에 있는지를 표현
    - FrozenLake 환경 (S : 출발, F : 길, H : 구멍, G : 목표지점)
        - SFFF
        - FHFH
        - FFFH
        - HFFG
        
- Discrete(4)(action_space)
    - 상하좌우로 이동하는 네 가지 행동이 있다는 뜻
    - 0 : Left
    - 1 : Down
    - 2 : Right
    - 3 : Up
- episode
    - 예) [1, 0.0, 4] : 행동 1을 하고 보상 0.0을 받고 상태 4로 전환

# 계산 모형

- 강화학습이 풀어야 하는 문제는 환경(environment)으로 정의됨
- 환경
    - 상태의 종류, 행동의 종류, 보상의 종류를 지정하며 행동을 취했을 때 발생하는 상태 변환을 지배하는 규칙을 포함하는 개념
    - 이런 정보를 통틀어서 마르코프 결정 프로세스(MDP) : Markov Decision Process)라고 함
    
- 상태와 행동, 보상
    - 에이전트와 환경은 밀접하게 상호작용하며, 에이전트는 환경이 제공하는 상태와 보상에 따라 행동을 취함
    - 슬롯머신 문제는 기계앞에서 도박을 하는 사람, FrozenLake 문제에서는 호수를 건너는 사람이 에이전트가 됨
    
- 슬롯머신 문제와 FrozenLake 문제를 MDP에 대입하면
    - 슬롯머신 문제
        - 상태 집합 : 공집합
        - 행동 집합 : {손잡이0, 손잡이1, 손잡이2, 손잡이3, 손잡이4}
        - 보상 집합 : {1, -1} (즉시 보상)
        
    - FroenLake 문제
        - 상태 집합 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}
        - 행동 집합 : {0(Left), 1(Down), 2(Right), 3(Up)}
        - 보상 집합 : {0, 1} (지연된 보상)
        
- 보상이 주어지는 시점
    - 슬롯머신 문제에서는 손잡이를 당겨 행동을 취하면 즉시 보상이 주어짐
    - FrozenLake에서는 목표 지점에 도달하면 1이라는 보상이 주어지는데, 그 외에는 0이 주어짐
        - 중간에 0이라는 보상이 주어지는 것은 보상 총액에 영향을 미치지 않기 때문에 의미가 없음
        - FrozenLake와 같은 경우를 지연된 보상(delayed reward)라고 함
        - 바둑과 장기도 지연된 보상 체계
        
- 상태 전이
    - 어떤 상태에서 행동을 취하면 새로운 상태로 전이
        - 예) FrozenLake 문제에서 1의 칸에서 2(Right)라는 행동을 취한다면
            - 주어지는 보상 : 0
            - 새로운 상태 : 2라는 칸으로 이동하기 때문에 2
                - 이 때 새로운 상태는 항상 2임
                - 100% 확률로 새로운 상태가 정해지는 환경을 결정론적 환경(deterministic environment)라고 함
                    - 바둑, 장기, 비디오 게임 등은 모두 결정론적 환경에 해당
                - FrozenLake 문제에서 얼음 바닥이 미끄러워 오른쪽 행동을 취했는데 다른 방향으로 이동하게 되는 상황을 허용한다면
                    - 스토케스틱 환경(stochastic environment)이 됨
                    - is_slippery = True로 설정하면 스토케스틱 환경이 됨

# 최적 정책

- 강화 학습은 누적 보상을 최대화하는 최적 정책을 알아내야 함
- 최적 정책
    - 슬롯머신 문제에서는 손잡이 3의 확률이 가장 높기 때문에 항상 손잡이 3을 당기게 됨
    - 정책은 확률분포로 표현할 수 있음
        - 현재 상태에서 어떤 행동을 취할지를 결정하는 확률 분포